# Data preparetion process

> Yes, it could be done in less amount of code, but that way it's much easier to read<br>
> Also it could be done line by line or even in Spark, but our datasets small enough to use them in Pandas

1. Get all dirs for files

In [1]:
import os
os.chdir('../Data/WHO Datasets/Unboxed')
cwd = os.getcwd()

In [2]:
# We are having datasets with and without metadata. So let's work only with 'DATA' marked datasets
datalist = []
for dirpath, dirname, filenames in os.walk(cwd):
    for file in filenames:
        if file.endswith("Data.csv"):
            datalist.append(os.path.join(dirpath, file))

2. Check what is in these datasets

In [3]:
with open(datalist[1], mode='r') as f:
    print(f.readline())
    print(f.readline())

Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]

"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Afghanistan,AFG,50.34,50.44

- Because all datasets are from WHO we are can use that as a reference for all of them
- Some datasets have more than 1 series, so for grouping will be much easier to use Pandas

3. Create a list of countries that you're have in any dataset

In [4]:
import pandas as pd

countries = set()

for dataset in datalist:
    df = pd.read_csv(dataset)
    df.drop(df.loc[df['Series Name'] != df['Series Name'].unique()[0]].index, axis=0, inplace=True)
    countries.update(df['Country Code'].tolist())

len(countries)

275

4. Create a list of countries that you're have in any dataset

In [5]:
country_check = {}
for country in countries:
    country_check[country] = 'Not'

for dataset in datalist:
    df = pd.read_csv(dataset)
    df.drop(df.loc[df['Series Name'] != df['Series Name'].unique()[0]].index, axis=0, inplace=True)
    country_list = df['Country Code'].tolist()

    for country in country_list:
        if country_check[country] != 'Delete':
            country_check[country] = 'Have'
    
    for country in country_check:
        if country_check[country] == 'Not':
            country_check[country] = 'Delete'
        elif country_check[country] == 'Have':
            country_check[country] = 'Not'

countries_filtered = set()

for country in country_check:
    if country_check[country] == 'Not':
        countries_filtered.add(country)

len(countries_filtered)

193

5. Get names of countries with void lines and delete them from our countries_filtered set

In [6]:
country_check = {}
info_cols = ['Series Name','Series Code','Country Name','Country Code']

for country in countries_filtered:
    country_check[country] = True

for dataset in datalist:
    df = pd.read_csv(dataset, na_values='..')
    df.drop(info_cols[0:3], axis=1, inplace=True)
    idx = list(set(df.index) - set(df.drop('Country Code', axis=1).dropna(how='all').index))
    for country in df.iloc[idx]['Country Code'].dropna().to_list():
        country_check[country] = False
    
for country in country_check:
    if country_check[country] == False and country in countries_filtered:
        countries_filtered.remove(country)

len(countries_filtered)

139

6. Choose a period that you're interested in

> Based on the fact that each dataset can cover different periods, let's use smallest timespan in our datasets

In [7]:
period = {}

for dataset in datalist:
    df = pd.read_csv(dataset, na_values='..')
    years = set(df.drop(info_cols, axis=1).dropna(how='all',axis=1).columns)
    print(df['Series Name'][0],'\n', years)

Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age) 
 {'2000 [YR2000]', '2010 [YR2010]', '2015 [YR2015]', '2005 [YR2005]', '2018 [YR2018]'}
Birth rate, crude (per 1,000 people) 
 {'1980 [YR1980]', '2001 [YR2001]', '1998 [YR1998]', '1981 [YR1981]', '1974 [YR1974]', '1975 [YR1975]', '1978 [YR1978]', '2018 [YR2018]', '1973 [YR1973]', '2000 [YR2000]', '2019 [YR2019]', '1967 [YR1967]', '2015 [YR2015]', '1989 [YR1989]', '1991 [YR1991]', '2005 [YR2005]', '1992 [YR1992]', '1964 [YR1964]', '2017 [YR2017]', '2020 [YR2020]', '1994 [YR1994]', '1969 [YR1969]', '2003 [YR2003]', '2009 [YR2009]', '1999 [YR1999]', '2002 [YR2002]', '1968 [YR1968]', '2016 [YR2016]', '1971 [YR1971]', '1979 [YR1979]', '1960 [YR1960]', '1997 [YR1997]', '1995 [YR1995]', '1972 [YR1972]', '1963 [YR1963]', '2008 [YR2008]', '1977 [YR1977]', '1961 [YR1961]', '1986 [YR1986]', '1990 [YR1990]', '2013 [YR2013]', '1987 [YR1987]', '1996 [YR1996]', '2011 [YR2011]', '1966 [YR1966]', '1970 

As we can see - we will have a very bad time with our datasets in some cases. So we should forget about datasets:<br>
`Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene`<br>
`Total alcohol consumption per capita`

In [8]:
del datalist[0]
del datalist[-4]

In [9]:
period = {}
years_min = 0
years_max = 9999

for dataset in datalist:
    df = pd.read_csv(dataset, na_values='..')
    years = set(df.drop(info_cols, axis=1).dropna(how='all',axis=1).columns)
    if int(min(years)[0:4]) > years_min:
        years_min = int(min(years)[0:4])
    if int(max(years)[0:4]) < years_max:
        years_max = int(max(years)[0:4])

print(years_min, years_max)

2000 2019


We will increase that timespan to 20 years. 1999-2019

In [10]:
# Savilg it as a set of columns
df = pd.read_csv(datalist[0], na_values='..')
years_col_list = df.drop(info_cols, axis=1).columns
years_col_list

Index(['1960 [YR1960]', '1961 [YR1961]', '1962 [YR1962]', '1963 [YR1963]',
       '1964 [YR1964]', '1965 [YR1965]', '1966 [YR1966]', '1967 [YR1967]',
       '1968 [YR1968]', '1969 [YR1969]', '1970 [YR1970]', '1971 [YR1971]',
       '1972 [YR1972]', '1973 [YR1973]', '1974 [YR1974]', '1975 [YR1975]',
       '1976 [YR1976]', '1977 [YR1977]', '1978 [YR1978]', '1979 [YR1979]',
       '1980 [YR1980]', '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]',
       '1984 [YR1984]', '1985 [YR1985]', '1986 [YR1986]', '1987 [YR1987]',
       '1988 [YR1988]', '1989 [YR1989]', '1990 [YR1990]', '1991 [YR1991]',
       '1992 [YR1992]', '1993 [YR1993]', '1994 [YR1994]', '1995 [YR1995]',
       '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]', '1999 [YR1999]',
       '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]',
       '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]',
       '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]', '2011 [YR2011]',
       '2012 [YR2012]', '

In [11]:
# We will check first and last position. We'll work with everything inbetween
years_col_list = [
    '2000 [YR2000]', '2019 [YR2019]'
]

In [12]:
years_col_list_full = [
    '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]',
    '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]',
    '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]', '2011 [YR2011]',
    '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]',
    '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]'
]

7. Delete all countries that have smaller observation period (SHOuLD BE REWRITTEN)

In [13]:
# For check if country is inside our period
countries_check = {}
for country in countries_filtered:
    countries_check[country] = 'Not'


for dataset in datalist:
    df = pd.read_csv(dataset, na_values='..')
    df_years = df.loc[df['Series Name'][0] == df['Series Name']].drop(info_cols[0:3], axis=1)

    df_years[years_col_list + ['Country Code']]
    
    for country in df_years[years_col_list + ['Country Code']].dropna(how='any')['Country Code']:
        if country in countries_check and countries_check[country] != 'Delete':
          countries_check[country] = 'Have'
    
    for country in countries_check:
        if countries_check[country] == 'Not':
            countries_check[country] = 'Delete'
        elif countries_check[country] == 'Have':
            countries_check[country] = 'Not'

b=[i[1] for i in countries_check.items()]

for k in list(set(b)):
    print("{0}: {1}".format(k, b.count(k)))

Not: 20
Delete: 119


So we will use 20 countries. Let's record list of their country codes in the 

In [14]:
for country in countries_check:
    if countries_check[country] == 'Delete' and country in countries_filtered:
        countries_filtered.remove(country)

len(countries_filtered)

20

8. Check for data gaps in your data and fill them if needed

In [15]:
len(datalist)

11

Datasets should be checked separatelty, one by one, each has it's own specific<br>
Since it will be a lot of datasets, here as functions i will create repeatetive tasks

In [16]:
df.loc[df['Series Name'][0] == df['Series Name']].drop(info_cols[0:3], axis=1)

,Country Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
0,RUS,43.5,36.815922,37.810944,36.318409,32.338310,30.769230,27.403847,22.115385,22.596153,...,19.248827,18.779343,20.689655,20.197044,17.733990,18.719212,18.840580,17.874395,20.289856,19.806763
1,AFG,1.0,0.497512,0.995025,9.452736,14.427860,15.384615,14.903846,17.788462,18.750000,...,14.084507,14.553990,16.256157,18.719212,20.689655,22.167488,20.289856,21.256039,19.806763,7.246377
2,ALB,29.5,38.805969,41.293533,48.258705,50.248756,49.038460,51.442307,49.038460,51.923077,...,50.704224,51.173710,50.246304,52.709358,52.216747,54.187191,52.657005,52.173912,51.207729,50.241547
3,DZA,14.5,12.437811,14.427860,18.905472,18.905472,23.557692,25.480770,22.596153,20.192308,...,22.535212,23.943663,25.123152,24.630543,23.645321,23.152710,21.256039,19.806763,19.323671,20.772947
4,ASM,NaN,NaN,NaN,NaN,NaN,64.423080,70.673080,65.384613,78.365387,...,80.751175,81.220657,NaN,NaN,NaN,NaN,95.169083,92.753624,81.642509,75.845413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,VIR,NaN,NaN,NaN,NaN,NaN,74.038460,85.576920,72.115387,70.673080,...,73.239433,72.300468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,PSE,24.5,28.855721,25.870647,24.378109,24.378109,26.923077,26.923077,33.173077,28.365385,...,20.657276,22.535212,22.660099,21.674877,18.719212,21.674877,22.222221,20.772947,17.391304,19.323671
211,YEM,27.5,26.368158,24.378109,14.925373,23.880596,20.192308,17.307692,16.346153,16.346153,...,9.389671,11.737089,11.822660,7.389163,5.418719,5.911330,3.864734,4.347826,4.347826,3.864734
212,ZMB,38.0,34.825871,35.820896,41.293533,38.805969,34.615383,34.615383,39.423077,39.903847,...,44.131454,44.131454,44.334976,45.812809,36.453201,36.453201,35.265701,36.231884,34.782608,35.265701


In [17]:
# Since it will be a lot of datasets, here as functions i will create a main repeatetive task
def who_dataset_check(dfloc, yearlist=[], countries=[], infcol=[]):
    df = pd.read_csv(dfloc, na_values="..")
    df = df.loc[df['Country Code'].isin(countries)]
    print('>', df['Series Name'].reset_index(drop=True)[0])

    # We will assume based on the data collection process that if table has more than 1 series, then gaps in them are positioned in the same places
    # It is based on the fact that each table is from the same research process. Different series is only separation by gender or class
    df_years = df.loc[df['Series Name'].reset_index(drop=True)[0] == df['Series Name']].drop(info_cols[0:3], axis=1)

    # For check if all years are in our dataset
    years_col_list_check = {}
    for year in yearlist:
        years_col_list_check[year] = False

    for col in df_years.columns.drop('Country Code'):
        if col in yearlist:
            years_col_list_check[col] = True

    # That list is needed for the column selection. If we're don't have any, we will get an error
    # I'm sure that there is another way, but this is easier
    temp_year_list = []
    for year in years_col_list_check:
        if years_col_list_check[year] == True:
            temp_year_list.append(year)
        else:
            print("Table doesn't have column {0}".format(year))

    #This is check if our dataset even has a null values
    na_all_check = False

    for country in df_years['Country Code']:
        row_check = False
        na_col_list = []
        for col in temp_year_list:
            row_check = df_years.loc[df['Country Code'] == country][col].isna().bool() or row_check
            if df_years.loc[df['Country Code'] == country][col].isna().bool() == True:
                na_col_list.append(col)
        if row_check == True:
            print('Country {0} has an null values in the next columns: {1}'.format(country, na_col_list))
            na_all_check = True

    # If row_check never triggered
    if na_all_check == False:
        print('Countries have no NaN values except missing columns')

    print()
    return None

In [18]:
for i in range(len(datalist)):
    print('Dataset', i)
    who_dataset_check(datalist[i], yearlist=years_col_list_full, countries=countries_filtered, infcol=info_cols)

Dataset 0
> Birth rate, crude (per 1,000 people)
Countries have no NaN values except missing columns

Dataset 1
> Control of Corruption: Percentile Rank
Table doesn't have column 2001 [YR2001]
Countries have no NaN values except missing columns

Dataset 2
> Death rate, crude (per 1,000 people)
Countries have no NaN values except missing columns

Dataset 3
> Domestic general government health expenditure per capita (current US$)
Countries have no NaN values except missing columns

Dataset 4
> GDP per capita (current US$)
Countries have no NaN values except missing columns

Dataset 5
> Income share held by highest 10%
Country ARG has an null values in the next columns: ['2015 [YR2015]']
Country AUT has an null values in the next columns: ['2001 [YR2001]', '2002 [YR2002]']
Country BEL has an null values in the next columns: ['2001 [YR2001]', '2002 [YR2002]']
Country COL has an null values in the next columns: ['2006 [YR2006]', '2007 [YR2007]']
Country DNK has an null values in the next co

9. Save in the most comfortable way